## Least Square (for $l=1$ Toy Model)

We successfully downloaded, loaded and filtered the data for 5 parameters solution, we defined the toroidal and spheroidal functions ($T_{lm}$ and $S_{lm}$), respectively modelling the right ascension (ra) and declination (dec). For $\alpha \in [0, 2\pi]$ and $\delta \in [-\pi/2,\pi/2]$ we visualised the VSH vector fields. We now want to perform a MLE on the dataset.

To do so, we will follow closely the procedure presented in the main paper (Gaia Early Data Release 3 Acceleration of the Solar System from Gaia astrometry). This assumes that the noise follows a Gaussian model, i.e. the astrometric measurement errors (in proper motion, parallax, etc.) are:
- Unbiased (zero mean),
- Independent between different sources (quasars),
- With known standard deviation and correlations, as provided in Gaia EDR3.

This allows the least-square estimation framework and the statistical significance tests, in particular using the $\chi^2$ distributions for assessing power in VSH.

Recall Eq. 5 amd 7:
$$
V(\alpha, \delta) = \sum_{l=1}^{l_{\text{max}}} \left( t_{l0} T_{l0} + s_{l0} S_{l0}
+ 2 \sum_{m=1}^{l} \left( t_{lm}^{\mathbb{R}} T_{lm}^{\mathbb{R}} - t_{lm}^{\mathbb{I}} T_{lm}^{\mathbb{I}} + s_{lm}^{\mathbb{R}} S_{lm}^{\mathbb{R}} - s_{lm}^{\mathbb{I}} S_{lm}^{\mathbb{I}} \right) \right)\tag{5}
$$

$$
X^2 = \begin{bmatrix}
\Delta\mu_{\alpha^*} & \Delta\mu_{\delta} \\
\end{bmatrix}
\begin{bmatrix}
\sigma_{\mu_{\alpha^*}}^2 & \rho_{\mu}\sigma_{\mu_{\alpha^*}}\sigma_{\mu_{\delta}} \\
\rho_{\mu}\sigma_{\mu_{\alpha^*}}\sigma_{\mu_{\delta}} & \sigma_{\mu_{\delta}}^2
\end{bmatrix}
\begin{bmatrix}
\Delta\mu_{\alpha^*} \\ \Delta\mu_{\delta} 
\end{bmatrix}\tag{7}
$$

where:
- $\Delta\mu_{\alpha^*} = \mu_{\alpha^* \text{obs}} - V_{\alpha^* \text{model}}$ is the difference between observed and predicted proper motion right ascension (ra).
- $\Delta\mu_{\delta} = \mu_{\delta \text{obs}} - V_{\delta \text{model}}$ is the difference between observed and predicted proper motion declination (dec).

Since each proper motion componet is assumed to follow a Gaussian distribution, MLE simplifies to a weighted least squares. Hence our objective is to minimise Eq. 7:

$$
\sum_k \begin{bmatrix}
\Delta\mu_{\alpha^*} & \Delta\mu_{\delta} \\
\end{bmatrix}
\begin{bmatrix}
\sigma_{\mu_{\alpha^*}}^2 & \rho_{\mu}\sigma_{\mu_{\alpha^*}}\sigma_{\mu_{\delta}} \\
\rho_{\mu}\sigma_{\mu_{\alpha^*}}\sigma_{\mu_{\delta}} & \sigma_{\mu_{\delta}}^2
\end{bmatrix}
\begin{bmatrix}
\Delta\mu_{\alpha^*} \\ \Delta\mu_{\delta} 
\end{bmatrix}
$$

In [1]:
import jax 
import jax.numpy as jnp
from jax import jit, vmap
from functools import partial, lru_cache
from src.models.vsh_model import*
from jax import random
import pandas as pd
from iminuit import Minuit # to perform least square
from src.models.configuration import*
from src.data.data_utils import load_qso_dataframe

/root/Document/Gaia_Project/mem97/gaia_venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load Fake Data
Before we proceed with the real dataset, we want to load the generated “fake” data to test our functions and minimiser.

In [2]:
vsh_coeff_gen = np.load('fake_data/vsh_gen_cov.npy') # Load true coefficient values

fake_data = pd.read_csv('fake_data/fake_vsh_data.csv')
angles_gen, obs_gen, error_gen = config_data(fake_data)
true_theta = np.load("fake_data/theta_true.npy")

In [11]:
# Bind fixed arguments into a new function
bound_least_square = partial(toy_least_square, angles_gen, obs_gen, error_gen)

# Now Minuit only sees the 6 free parameters
m = Minuit(bound_least_square,
           t_10=0.0, t_11r=0.0, t_11i=0.0,
           s_10=0.0, s_11r=0.0, s_11i=0.0)

m.errordef=Minuit.LEAST_SQUARES
lim = 0.1
m.limits['t_10'] = (-lim,lim)
m.limits['t_11r'] = (-lim,lim)
m.limits['t_11i'] = (-lim,lim)
m.limits['s_10'] = (-lim,lim)
m.limits['s_11r'] = (-lim,lim)
m.limits['s_11i'] = (-lim,lim)

m.migrad()

┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = 2.371e+05                  │             Nfcn = 1082              │
│ EDM = 0.344 (Goal: 0.0002)       │            time = 3.3 sec            │
├──────────────────────────────────┼──────────────────────────────────────┤
│         INVALID Minimum          │   ABOVE EDM threshold (goal x 10)    │
├──────────────────────────────────┼──────────────────────────────────────┤
│      No parameters at limit      │           Below call limit           │
├──────────────────────────────────┼──────────────────────────────────────┤
│             Hesse ok             │         Covariance accurate          │
└──────────────────────────────────┴──────────────────────────────────────┘
┌───┬───────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name  │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼───────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ t_10  │ 42.77e-3  │  0.25e-3  │            │            │  -0.1   │   0.1   │       │
│ 1 │ t_11r │  22.6e-3  │  0.5e-3   │            │            │  -0.1   │   0.1   │       │
│ 2 │ t_11i │  11.0e-3  │  0.4e-3   │            │            │  -0.1   │   0.1   │       │
│ 3 │ s_10  │ 93.91e-3  │  0.28e-3  │            │            │  -0.1   │   0.1   │       │
│ 4 │ s_11r │  36.6e-3  │  0.5e-3   │            │            │  -0.1   │   0.1   │       │
│ 5 │ s_11i │ -63.5e-3  │  0.4e-3   │            │            │  -0.1   │   0.1   │       │
└───┴───────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘
┌───────┬───────────────────────────────────────────────────────┐
│       │     t_10    t_11r    t_11i     s_10    s_11r    s_11i │
├───────┼───────────────────────────────────────────────────────┤
│  t_10 │ 6.21e-08        0        0  0.02e-6        0  0.02e-6 │
│ t_11r │        0 2.18e-07        0        0        0        0 │
│ t_11i │        0        0 1.84e-07        0        0        0 │
│  s_10 │  0.02e-6        0        0 7.66e-08        0 -0.01e-6 │
│ s_11r │        0        0        0        0 2.18e-07        0 │
│ s_11i │  0.02e-6        0        0 -0.01e-6        0 1.75e-07 │
└───────┴───────────────────────────────────────────────────────┘

Quick comparison to predicted vs true parameters values

In [12]:
theta_fit = jnp.array([m.values[k] for k in m.parameters])
print("Fitted parameters values:")
print(theta_fit)
print("True values:")
print(true_theta[:count_vsh_coeffs(1)])

Fitted parameters values:
[ 0.04276923  0.02264982  0.01101284  0.09390939  0.03662318 -0.06349312]
True values:
[ 0.05372004  0.05742959 -0.02012502 -0.00375978  0.00838665 -0.04013964]


# Toy Model on True Dataset
Now that we have verified that the minimisation of the least square is working correctly on the generated dataset, we can use the true dataset.

In [2]:
df = load_qso_dataframe()
angles, obs, error = config_data(df)

In [20]:
# Bind fixed arguments into a new function
bound_least_square = partial(toy_least_square, angles, obs, error)

# Now Minuit only sees the 6 free parameters
m = Minuit(bound_least_square,
           t_10=0.0, t_11r=0.0, t_11i=0.0,
           s_10=0.0, s_11r=0.0, s_11i=0.0)

m.errordef=Minuit.LEAST_SQUARES
lim = 0.05
m.limits['t_10'] = (-lim,lim)
m.limits['t_11r'] = (-lim,lim)
m.limits['t_11i'] = (-lim,lim)
m.limits['s_10'] = (-lim,lim)
m.limits['s_11r'] = (-lim,lim)
m.limits['s_11i'] = (-lim,lim)

m.migrad()

┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = 2.728e+06                  │              Nfcn = 123              │
│ EDM = 0 (Goal: 0.0002)           │            time = 1.9 sec            │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Valid Minimum           │   Below EDM threshold (goal x 10)    │
├──────────────────────────────────┼──────────────────────────────────────┤
│      No parameters at limit      │           Below call limit           │
├──────────────────────────────────┼──────────────────────────────────────┤
│             Hesse ok             │         Covariance accurate          │
└──────────────────────────────────┴──────────────────────────────────────┘
┌───┬───────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name  │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼───────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ t_10  │  -3.4e-3  │  0.9e-3   │            │            │  -0.05  │  0.05   │       │
│ 1 │ t_11r │  -0.0102  │  0.0011   │            │            │  -0.05  │  0.05   │       │
│ 2 │ t_11i │  0.0057   │  0.0011   │            │            │  -0.05  │  0.05   │       │
│ 3 │ s_10  │  -5.2e-3  │  0.8e-3   │            │            │  -0.05  │  0.05   │       │
│ 4 │ s_11r │  0.0006   │  0.0012   │            │            │  -0.05  │  0.05   │       │
│ 5 │ s_11i │  -0.0208  │  0.0012   │            │            │  -0.05  │  0.05   │       │
└───┴───────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘
┌───────┬───────────────────────────────────────────────────────┐
│       │     t_10    t_11r    t_11i     s_10    s_11r    s_11i │
├───────┼───────────────────────────────────────────────────────┤
│  t_10 │ 8.37e-07   0.5e-6   0.1e-6   0.2e-6        0   0.5e-6 │
│ t_11r │   0.5e-6 1.19e-06   0.2e-6   0.3e-6   0.1e-6   0.5e-6 │
│ t_11i │   0.1e-6   0.2e-6 1.17e-06  -0.1e-6  -0.2e-6   0.3e-6 │
│  s_10 │   0.2e-6   0.3e-6  -0.1e-6 6.66e-07        0   0.3e-6 │
│ s_11r │        0   0.1e-6  -0.2e-6        0 1.39e-06  -0.2e-6 │
│ s_11i │   0.5e-6   0.5e-6   0.3e-6   0.3e-6  -0.2e-6 1.43e-06 │
└───────┴───────────────────────────────────────────────────────┘

Checking results for $l=1$

In [21]:
m.params

,Name,Value,Hesse Error,Minos Error-,Minos Error+,Limit-,Limit+,Fixed
0,t_10,-3.4e-3,0.9e-3,,,-0.05,0.05,
1,t_11r,-0.0102,0.0011,,,-0.05,0.05,
2,t_11i,0.0057,0.0011,,,-0.05,0.05,
3,s_10,-5.2e-3,0.8e-3,,,-0.05,0.05,
4,s_11r,0.0006,0.0012,,,-0.05,0.05,
5,s_11i,-0.0208,0.0012,,,-0.05,0.05,


In [19]:
parameters = jnp.array([m.values[k] for k in m.parameters])
hessian_matrix = m.covariance
variances = jnp.diag(hessian_matrix)
spheroidal_vector_summary(parameters, variances, index = jnp.array([3,4,5]))
print('')
toroidal_vector_summary(parameters, variances, index = jnp.array([0,1,2]))

Equatorial components:
G_vec = [ -0.2577563 -10.189044   -1.7785045] +/- [4.866633  5.5721984 2.3813047](μas/yr)
Magnitude = 10.34631061553955 +/- 11.296056747436523 (μas/yr)
RA = 268.5508728027344 +/- 55.99720764160156 (deg)
Dec = -9.898159980773926 +/- 0.4520724415779114 (deg)

Galactic components:
G_vec = [9.77413645 3.07651773 1.43088618] +/- [5.00860627 3.88462222 4.49752138](μas/yr)
l = 17.471972965281086 +/- 22.36007447514229 (deg)
d = 7.949438822904455 +/- 3.6677001043576833 (deg)

R_vec = [ 4.9159555 -2.7544324 -1.1734687] +/- [5.020394  3.7802556 2.5087306](μas/yr)
Magnitude = 5.755913734436035 +/- 9.638985633850098 (μas/yr)
RA = 330.7378845214844 +/- 85.54389953613281 (deg)
Dec = -11.763463973999023 +/- 18.195499420166016 (deg)


In [22]:
hessian_matrix

,t_10,t_11r,t_11i,s_10,s_11r,s_11i
t_10,5.27e-05,0.07e-3 (0.994),0.06e-3 (0.991),0.05e-3 (0.993),0.07e-3 (0.992),0.08e-3 (0.993)
t_11r,0.07e-3 (0.994),0.000106,0.08e-3 (0.992),0.07e-3 (0.994),0.10e-3 (0.993),0.12e-3 (0.995)
t_11i,0.06e-3 (0.991),0.08e-3 (0.992),5.99e-05,0.05e-3 (0.991),0.08e-3 (0.990),0.09e-3 (0.992)
s_10,0.05e-3 (0.993),0.07e-3 (0.994),0.05e-3 (0.991),4.75e-05,0.07e-3 (0.994),0.08e-3 (0.995)
s_11r,0.07e-3 (0.992),0.10e-3 (0.993),0.08e-3 (0.990),0.07e-3 (0.994),9.92e-05,0.11e-3 (0.993)
s_11i,0.08e-3 (0.993),0.12e-3 (0.995),0.09e-3 (0.992),0.08e-3 (0.995),0.11e-3 (0.993),0.00013


In [23]:
cov_slm, cov_tlm, full_cov = cov_matrix_minuit(hessian_matrix,
                            indices_s=jnp.array([4,5,3]),
                            indices_t=jnp.array([1,2,0]))
print(full_cov.shape)
print(cov_slm.shape)
print(cov_tlm.shape)

(6, 6)
(3, 3)
(3, 3)


In [24]:
cov_slm

array([[9.92077972e-05, 1.12801955e-04, 6.82176318e-05],
       [1.12801955e-04, 1.30059402e-04, 7.81802335e-05],
       [6.82176318e-05, 7.81802335e-05, 4.75060122e-05]])

In [25]:
rho_slm = rho_matrix(cov_slm)
print(rho_slm)

[[1.         0.99305396 0.99368663]
 [0.99305396 1.         0.99460766]
 [0.99368663 0.99460766 1.        ]]


In [28]:
h = jnp.array(hessian_matrix)
print(h)

[[5.27262200e-05 7.41855547e-05 5.56545128e-05 4.96949324e-05
  7.17516887e-05 8.22520160e-05]
 [7.41855547e-05 1.05575768e-04 7.88425314e-05 7.04066115e-05
  1.01647907e-04 1.16602154e-04]
 [5.56545128e-05 7.88425314e-05 5.98592051e-05 5.28628952e-05
  7.63269927e-05 8.74847974e-05]
 [4.96949324e-05 7.04066115e-05 5.28628952e-05 4.75060115e-05
  6.82176324e-05 7.81802300e-05]
 [7.17516887e-05 1.01647907e-04 7.63269927e-05 6.82176324e-05
  9.92077985e-05 1.12801958e-04]
 [8.22520160e-05 1.16602154e-04 8.74847974e-05 7.81802300e-05
  1.12801958e-04 1.30059401e-04]]


In [33]:
cov_tlm = cov_ts(h, np.array([0,1,2]))
cov_slm = cov_ts(h, [3,4,5])
print(cov_tlm)
print(cov_slm)

[[5.2726220e-05 7.4185555e-05 5.5654513e-05]
 [7.4185555e-05 1.0557577e-04 7.8842531e-05]
 [5.5654513e-05 7.8842531e-05 5.9859205e-05]]
[[4.7506011e-05 6.8217632e-05 7.8180230e-05]
 [6.8217632e-05 9.9207798e-05 1.1280196e-04]
 [7.8180230e-05 1.1280196e-04 1.3005940e-04]]


# For Albitrary Choice of $l_{max}$

$l=2$ on generated data

In [45]:
def eval_loss_gendata(theta, lmax):
    return least_square(angles_gen, obs_gen, error_gen, theta, lmax=lmax, grid=False)
eval_loss_gendata = jit(eval_loss_gendata, static_argnames=['lmax'])

def run_iminuit_gendata(lmax, t_bound, s_bound):

    def least_square_wrapper(*theta_flat):
        theta = jnp.array(theta_flat)
        return float(eval_loss_gendata(theta, lmax)) 

    total_params = count_vsh_coeffs(lmax) 
    limits = vsh_minuit_limits(lmax, t_bound=t_bound, s_bound=s_bound)
    
    theta_init = jnp.zeros(total_params)    

    m = Minuit(least_square_wrapper, *theta_init)

    m.errordef = Minuit.LEAST_SQUARES
    for i, name in enumerate(m.parameters):
        m.limits[name] = limits[name]

    m.migrad()

    return m

In [46]:
ml2 = run_iminuit_gendata(2, 0.08, 0.08)

ml2.migrad()
print(ml2.params)
print("Is the function minimum valid?",m.fmin.is_valid)

┌───┬──────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼──────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ x0   │  0.0524   │  0.0018   │            │            │  -0.08  │  0.08   │       │
│ 1 │ x1   │  55.4e-3  │  0.4e-3   │            │            │  -0.08  │  0.08   │       │
│ 2 │ x2   │ -21.1e-3  │  0.9e-3   │            │            │  -0.08  │  0.08   │       │
│ 3 │ x3   │  -3.6e-3  │  1.0e-3   │            │            │  -0.08  │  0.08   │       │
│ 4 │ x4   │  0.0125   │  0.0023   │            │            │  -0.08  │  0.08   │       │
│ 5 │ x5   │ -39.1e-3  │  1.0e-3   │            │            │  -0.08  │  0.08   │       │
│ 6 │ x6   │  -0.0167  │  0.0023   │            │            │  -0.08  │  0.08   │       │
│ 7 │ x7   │  0.0231   │  0.0016   │            │            │  -0.08  │  0.08   │       │

In [48]:
theta_fit = jnp.array([ml2.values[k] for k in ml2.parameters])
print("Fitted parameters values:")
print(theta_fit)
print("True values:")
print(true_theta)

Fitted parameters values:
[ 0.05239322  0.05544324 -0.02108251 -0.00360312  0.0125017  -0.03907218
 -0.01674366  0.02309791  0.03179578 -0.03509466  0.05917309 -0.0602842
  0.01481551  0.00678882  0.04491844  0.05286903]
True values:
[ 0.05372004  0.05742959 -0.02012502 -0.00375978  0.00838665 -0.04013964
 -0.02277664  0.02273766  0.02961199 -0.03947825  0.05824246 -0.05696608
  0.01680502  0.0075229   0.04790566  0.0521445 ]


# On Real Data for $l=1$


In [3]:
# Load data
df = load_qso_dataframe()
angles, obs, error = config_data(df)

In [31]:
lmax = 1
total_params = count_vsh_coeffs(lmax) 
limits = vsh_minuit_limits(lmax, t_bound=0.01, s_bound=0.0025)

# Flat vector theta: [t10, ..., t_lmaxm, s10, ..., s_lmaxm]
theta_init = jnp.zeros(total_params)
#theta_init = jnp.array([0., -0.07, 0., 0.0, 0.0006, -0.005])

# Fix everything except theta
#bound_least_square = partial(least_square, data, obs, error, lmax=lmax, grid=False)

def least_square_wrapper(*theta_flat):
    theta = jnp.array(theta_flat)  # reconstructs the vector from scalars
    return least_square(angles, obs, error, theta, lmax=lmax, grid=False)


m = Minuit(least_square_wrapper, *theta_init)

m.errordef = Minuit.LEAST_SQUARES
for i, name in enumerate(m.parameters):
    m.limits[name] = limits[name]


m.migrad()
m.params
print(m.params)
print('Converged to miminum?', m.fmin.is_valid)

┌───┬──────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼──────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ x0   │  -3.0e-3  │  0.9e-3   │            │            │  -0.01  │  0.01   │       │
│ 1 │ x1   │ -2.50e-3  │  0.11e-3  │            │            │ -0.0025 │ 0.0025  │       │
│ 2 │ x2   │ -10.0e-3  │  0.7e-3   │            │            │  -0.01  │  0.01   │       │
│ 3 │ x3   │  0.0055   │  0.0012   │            │            │  -0.01  │  0.01   │       │
│ 4 │ x4   │  -0.0000  │  0.0012   │            │            │ -0.0025 │ 0.0025  │       │
│ 5 │ x5   │ -2.500e-3 │ 0.035e-3  │            │            │ -0.0025 │ 0.0025  │       │
└───┴──────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘
Converged to miminum? False


In [32]:
m

┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = 2.728e+06                  │             Nfcn = 3864              │
│ EDM = 7.94 (Goal: 0.0002)        │           time = 52.3 sec            │
├──────────────────────────────────┼──────────────────────────────────────┤
│         INVALID Minimum          │   ABOVE EDM threshold (goal x 10)    │
├──────────────────────────────────┼──────────────────────────────────────┤
│     SOME parameters at limit     │           Below call limit           │
├──────────────────────────────────┼──────────────────────────────────────┤
│             Hesse ok             │         Covariance accurate          │
└──────────────────────────────────┴──────────────────────────────────────┘
┌───┬──────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼──────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ x0   │  -3.0e-3  │  0.9e-3   │            │            │  -0.01  │  0.01   │       │
│ 1 │ x1   │ -2.50e-3  │  0.11e-3  │            │            │ -0.0025 │ 0.0025  │       │
│ 2 │ x2   │ -10.0e-3  │  0.7e-3   │            │            │  -0.01  │  0.01   │       │
│ 3 │ x3   │  0.0055   │  0.0012   │            │            │  -0.01  │  0.01   │       │
│ 4 │ x4   │  -0.0000  │  0.0012   │            │            │ -0.0025 │ 0.0025  │       │
│ 5 │ x5   │ -2.500e-3 │ 0.035e-3  │            │            │ -0.0025 │ 0.0025  │       │
└───┴──────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘
┌────┬───────────────────────────────────────────────────────┐
│    │       x0       x1       x2       x3       x4       x5 │
├────┼───────────────────────────────────────────────────────┤
│ x0 │ 8.81e-07   0.8e-9  13.4e-9  -0.1e-6       -0 0.061e-9 │
│ x1 │   0.8e-9    9e-10   0.1e-9  -0.1e-9        0 0.003e-9 │
│ x2 │  13.4e-9   0.1e-9 3.26e-09  -0.7e-9   0.1e-9 0.008e-9 │
│ x3 │  -0.1e-6  -0.1e-9  -0.7e-9 1.34e-06   0.1e-6       -0 │
│ x4 │       -0        0   0.1e-9   0.1e-6 1.57e-06 0.208e-9 │
│ x5 │ 0.061e-9 0.003e-9 0.008e-9       -0 0.208e-9 1.59e-11 │
└────┴───────────────────────────────────────────────────────┘

In [33]:
parameters = jnp.array([m.values[k] for k in m.parameters])
hessian_matrix = m.covariance
variances = jnp.diag(hessian_matrix)
spheroidal_vector_summary(parameters, variances)
print('')
toroidal_vector_summary(parameters, variances)

Equatorial components:
G_vec = [ 0.00783932 -1.2214506  -0.86302274] +/- [0.61163926 0.00194626 0.01036628](μas/yr)
Magnitude = 1.495597243309021 +/- 0.018798954784870148 (μas/yr)
RA = 270.36773681640625 +/- 58.71763229370117 (deg)
Dec = -35.242774963378906 +/- 0.17760618031024933 (deg)

Galactic components:
G_vec = [ 1.48399062 -0.09745181 -0.1583858 ] +/- [0.03397927 0.30231717 0.5307199 ](μas/yr)
l = 356.24285231286177 +/- 11.622439553872443 (deg)
d = -6.079100816040035 +/- 0.14080667612608858 (deg)

R_vec = [ 4.8854294 -2.7006495 -1.0269868] +/- [0.02791138 0.56492174 0.32431722](μas/yr)
Magnitude = 5.675881862640381 +/- 0.5778511762619019 (μas/yr)
RA = 331.0663146972656 +/- 10.389854431152344 (deg)
Dec = -10.424442291259766 +/- 0.10509473085403442 (deg)


In [34]:
print(f's_10 = {parameters[1]}, s_11r = {parameters[4]}, s_11i = {parameters[5]}')

s_10 = -0.002497937297448516, s_11r = -1.6044381482061e-05, s_11i = -0.0024998860899358988


# On Real Data for $l=2$

In [15]:
lmax = 2
total_params = count_vsh_coeffs(lmax) 
limits = vsh_minuit_limits(lmax=2, t_bound=0.05, s_bound=0.01)

# Flat vector theta: [t10, ..., t_lmaxm, s10, ..., s_lmaxm]
theta_init = jnp.zeros(total_params)

# Fix everything except theta
#bound_least_square = partial(least_square, data, obs, error, lmax=lmax, grid=False)

def least_square_wrapper(*theta_flat):
    theta = jnp.array(theta_flat)  # reconstructs the vector from scalars
    return least_square(angles, obs, error, theta, lmax=lmax, grid=False)


m = Minuit(least_square_wrapper, *theta_init)

m.errordef = Minuit.LEAST_SQUARES
for i, name in enumerate(m.parameters):
    m.limits[name] = limits[name]


m.migrad()
print(m.params)
print('Converged to miminum?', m.fmin.is_valid)

┌───┬──────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼──────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ x0   │  -0.002   │   0.001   │            │            │  -0.05  │  0.05   │       │
│ 1 │ x1   │  -6.4e-3  │  0.9e-3   │            │            │  -0.01  │  0.01   │       │
│ 2 │ x2   │  -0.0113  │  0.0015   │            │            │  -0.05  │  0.05   │       │
│ 3 │ x3   │  0.0050   │  0.0015   │            │            │  -0.05  │  0.05   │       │
│ 4 │ x4   │  0.0028   │  0.0021   │            │            │  -0.01  │  0.01   │       │
│ 5 │ x5   │ -10.00e-3 │  0.12e-3  │            │            │  -0.01  │  0.01   │       │
│ 6 │ x6   │  0.0061   │  0.0012   │            │            │  -0.05  │  0.05   │       │
│ 7 │ x7   │  -3.4e-3  │  0.9e-3   │            │            │  -0.01  │  0.01   │       │

In [13]:
parameters = jnp.array([m.values[k] for k in m.parameters])
hessian_matrix = m.covariance
variances = jnp.diag(hessian_matrix)
spheroidal_vector_summary(parameters, variances)
print('')
toroidal_vector_summary(parameters, variances)

Equatorial components:
G_vec = [-1.4890867 -4.1518087 -2.152283 ] +/- [0.6835167  0.0151833  0.27026516](μas/yr)
Magnitude = 4.907873153686523 +/- 0.5463750958442688 (μas/yr)
RA = 250.26913452148438 +/- 17.10559844970703 (deg)
Dec = -26.010560989379883 +/- 2.4181487560272217 (deg)

Galactic components:
G_vec = [ 4.74940793 -0.49664152  1.13299925] +/- [0.13668177 0.39352941 0.60574047](μas/yr)
l = 354.03032523434655 +/- 4.699195290172857 (deg)
d = 13.347321178114218 +/- 0.3678359591331978 (deg)

R_vec = [ 5.643859  -2.4430923 -0.7092557] +/- [0.6356071  0.6150645  0.29733017](μas/yr)
Magnitude = 6.190710067749023 +/- 1.2895753383636475 (μas/yr)
RA = 336.5933532714844 +/- 11.790470123291016 (deg)
Dec = -6.57869291305542 +/- 1.2667852640151978 (deg)


In [14]:
m

┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = 2.728e+06                  │             Nfcn = 3017              │
│ EDM = 3.43 (Goal: 0.0002)        │           time = 44.2 sec            │
├──────────────────────────────────┼──────────────────────────────────────┤
│         INVALID Minimum          │   ABOVE EDM threshold (goal x 10)    │
├──────────────────────────────────┼──────────────────────────────────────┤
│     SOME parameters at limit     │           Below call limit           │
├──────────────────────────────────┼──────────────────────────────────────┤
│             Hesse ok             │         Covariance accurate          │
└──────────────────────────────────┴──────────────────────────────────────┘
┌───┬──────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼──────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ x0   │  -2.1e-3  │  0.9e-3   │            │            │  -0.05  │  0.05   │       │
│ 1 │ x1   │  -6.2e-3  │  0.8e-3   │            │            │ -0.0085 │ 0.0085  │       │
│ 2 │ x2   │  -0.0116  │  0.0013   │            │            │  -0.05  │  0.05   │       │
│ 3 │ x3   │  0.0050   │  0.0013   │            │            │  -0.05  │  0.05   │       │
│ 4 │ x4   │  0.0030   │  0.0014   │            │            │ -0.0085 │ 0.0085  │       │
│ 5 │ x5   │ -8.50e-3  │  0.09e-3  │            │            │ -0.0085 │ 0.0085  │       │
│ 6 │ x6   │  6.5e-3   │  0.8e-3   │            │            │  -0.05  │  0.05   │       │
│ 7 │ x7   │  -3.6e-3  │  0.9e-3   │            │            │ -0.0085 │ 0.0085  │       │
│ 8 │ x8   │  0.0014   │  0.0014   │            │            │  -0.05  │  0.05   │       │
│ 9 │ x9   │  0.0037   │  0.0014   │            │            │  -0.05  │  0.05   │       │
│ 10│ x10  │  0.0073   │  0.0010   │            │            │ -0.0085 │ 0.0085  │       │
│ 11│ x11  │  0.0070   │  0.0012   │            │            │ -0.0085 │ 0.0085  │       │
│ 12│ x12  │  -4.9e-3  │  1.0e-3   │            │            │  -0.05  │  0.05   │       │
│ 13│ x13  │  0.0050   │  0.0013   │            │            │  -0.05  │  0.05   │       │
│ 14│ x14  │  -0.0048  │  0.0013   │            │            │ -0.0085 │ 0.0085  │       │
│ 15│ x15  │  0.0058   │  0.0013   │            │            │ -0.0085 │ 0.0085  │       │
└───┴──────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘
┌─────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│     │       x0       x1       x2       x3       x4       x5       x6       x7       x8       x9      x10      x11      x12      x13      x14      x15 │
├─────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│  x0 │ 7.41e-07        0   0.2e-6  -0.2e-6   0.1e-6   1.9e-9        0   0.1e-6  -0.1e-6       -0   0.1e-6   0.2e-6   0.2e-6        0   0.1e-6   0.2e-6 │
│  x1 │        0 6.12e-07   0.1e-6       -0        0       -0       -0        0  -0.2e-6  -0.2e-6        0       -0        0  -0.2e-6       -0        0 │
│  x2 │   0.2e-6   0.1e-6 1.69e-06        0       -0  -0.2e-9        0  -0.2e-6  -0.1e-6       -0  -0.1e-6  -0.1e-6  -0.1e-6  -0.1e-6  -0.2e-6  -0.3e-6 │
│  x3 │  -0.2e-6       -0        0 1.58e-06       -0  -0.6e-9       -0  -0.4e-6        0       -0  -0.2e-6  -0.1e-6  -0.1e-6       -0   0.3e-6  -0.1e-6 │
│  x4 │   0.1e-6        0       -0       -0 1.96e-06     2e-9   0.3e-6   0.1e-6       -0   0.4e-6   0.1e-6   0.1e-6   0.3e-6   0.4e-6       -0   0.1e-6 │
│  x5 │   1.9e-9       -0  -0.2e-9  

## On Real Data for l=3

In [3]:
lmax = 3
total_params = count_vsh_coeffs(lmax) 
limits = vsh_minuit_limits(lmax=3, t_bound=0.05, s_bound=0.01)

# Flat vector theta: [t10, ..., t_lmaxm, s10, ..., s_lmaxm]
theta_init = jnp.zeros(total_params)

# Fix everything except theta
#bound_least_square = partial(least_square, data, obs, error, lmax=lmax, grid=False)

def least_square_wrapper(*theta_flat):
    theta = jnp.array(theta_flat)  # reconstructs the vector from scalars
    return least_square(angles, obs, error, theta, lmax=lmax, grid=False)


m = Minuit(least_square_wrapper, *theta_init)

m.errordef = Minuit.LEAST_SQUARES
for i, name in enumerate(m.parameters):
    m.limits[name] = limits[name]


m.migrad()
print(m.params)
print('Converged to miminum?', m.fmin.is_valid)

┌───┬──────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼──────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ x0   │  -0.1e-3  │  0.8e-3   │            │            │  -0.05  │  0.05   │       │
│ 1 │ x1   │  -8.4e-3  │  0.7e-3   │            │            │  -0.01  │  0.01   │       │
│ 2 │ x2   │  -0.0113  │  0.0019   │            │            │  -0.05  │  0.05   │       │
│ 3 │ x3   │  0.0072   │  0.0011   │            │            │  -0.05  │  0.05   │       │
│ 4 │ x4   │   0.001   │   0.004   │            │            │  -0.01  │  0.01   │       │
│ 5 │ x5   │ -10.0e-3  │  0.6e-3   │            │            │  -0.01  │  0.01   │       │
│ 6 │ x6   │  0.0066   │  0.0020   │            │            │  -0.05  │  0.05   │       │
│ 7 │ x7   │  -0.0043  │  0.0018   │            │            │  -0.01  │  0.01   │       │

In [4]:
parameters = jnp.array([m.values[k] for k in m.parameters])
hessian_matrix = m.covariance
variances = jnp.diag(hessian_matrix)
spheroidal_vector_summary(parameters, variances)
print('')
toroidal_vector_summary(parameters, variances)

Equatorial components:
G_vec = [-0.67166525 -4.8808007  -2.893719  ] +/- [1.859902   0.07742357 0.23213719](μas/yr)
Magnitude = 5.713751792907715 +/- 0.57822585105896 (μas/yr)
RA = 262.1645202636719 +/- 43.85564041137695 (deg)
Dec = -30.42752456665039 +/- 2.6356284618377686 (deg)

Galactic components:
G_vec = [ 5.70001302 -0.32230793  0.23006366] +/- [0.16614718 0.93584537 1.61731451](μas/yr)
l = 356.76364951027665 +/- 9.377485231194852 (deg)
d = 2.3076327094806945 +/- 0.11421469191176485 (deg)

R_vec = [ 5.540399   -3.5118     -0.02007375] +/- [0.92414874 0.5405345  0.26897582](μas/yr)
Magnitude = 6.559661865234375 +/- 1.7037744522094727 (μas/yr)
RA = 327.63128662109375 +/- 12.034881591796875 (deg)
Dec = -0.17533572018146515 +/- 0.061749767512083054 (deg)


In [5]:
chi2_red(m, 3)

Goodness of fit χ^2_red = 1.121628506575214


### Higher $l_{max}$ values

In [ ]:
lmax = 10
total_params = count_vsh_coeffs(lmax) 
limits = vsh_minuit_limits(lmax=lmax, t_bound=0.05, s_bound=0.01)

# Flat vector theta: [t10, ..., t_lmaxm, s10, ..., s_lmaxm]
theta_init = jnp.zeros(total_params)

# Fix everything except theta
#bound_least_square = partial(least_square, data, obs, error, lmax=lmax, grid=False)

def least_square_wrapper(*theta_flat):
    theta = jnp.array(theta_flat)  # reconstructs the vector from scalars
    return least_square(angles, obs, error, theta, lmax=lmax, grid=False)


m = Minuit(least_square_wrapper, *theta_init)

m.errordef = Minuit.LEAST_SQUARES
for i, name in enumerate(m.parameters):
    m.limits[name] = limits[name]


m.migrad()
print(m.params)
print('Converged to miminum?', m.fmin.is_valid)

In [7]:
parameters = jnp.array([m.values[k] for k in m.parameters])
hessian_matrix = m.covariance
variances = jnp.diag(hessian_matrix)
spheroidal_vector_summary(parameters, variances)
print('')
toroidal_vector_summary(parameters, variances)

Equatorial components:
G_vec = [-0.57907367 -4.8839927  -2.6446667 ] +/- [0.8276999 0.0235049 0.3917958](μas/yr)
Magnitude = 5.584171295166016 +/- 0.5666340589523315 (μas/yr)
RA = 263.2382507324219 +/- 19.59767723083496 (deg)
Dec = -28.268268585205078 +/- 0.9705260992050171 (deg)

Galactic components:
G_vec = [ 5.57721953 -0.08910026  0.26392104] +/- [0.19600821 0.50301276 0.74006926](μas/yr)
l = 359.0847348369106 +/- 5.166322127184473 (deg)
d = 2.708941994263007 +/- 0.10402993811961156 (deg)

R_vec = [ 5.333627   -3.0461373   0.05343694] +/- [0.7908783  0.6525923  0.38781893](μas/yr)
Magnitude = 6.142425537109375 +/- 1.5538028478622437 (μas/yr)
RA = 330.26849365234375 +/- 13.157622337341309 (deg)
Dec = 0.4984593987464905 +/- 0.13094159960746765 (deg)


In [8]:
chi2_red(m, 4)

Goodness of fit χ^2_red = 1.1215534384606907
